# Imports

In [1]:
import pandas as pd
import glob
import ast
import requests
from bs4 import BeautifulSoup
import re

# Read in News Articles

In [13]:
#define glob of files
path = r'./data' 
all_files = glob.glob(path + "/*.csv")
#iterate over files
filelist= []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    #append dataframe from file to list
    filelist.append(df)
#concatenate list of dataframes
df = pd.concat(filelist, axis=0, ignore_index=True)

/Users/johanneshuessy/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [14]:
#Scrape wiki article for cabinet members
base_url='https://en.wikipedia.org/wiki/Cabinet_of_the_United_States'
req=requests.get(base_url)
cab_soup = BeautifulSoup(req.content, 'html.parser')
#create list of cabinet members
cabinet = [cab_soup.find_all('table')[3].find_all('tr')[i].find_all('td')[1].find_all('a')[-1]['title']
 for i in range(1,len(cab_soup.find_all('table')[3].find_all('tr')))]
#scrape wiki article for senators
base_url='https://en.wikipedia.org/wiki/List_of_current_United_States_Senators'
req=requests.get(base_url)
sen_soup = BeautifulSoup(req.content, 'html.parser')
#create list of senators
senate=[sen_soup.find_all('table')[4].tbody.find_all('tr')[i].find_all('th')[0].find_all('a')[0]['title']
 for i in range(1,len(sen_soup.find_all('table')[4].find_all('tr')))]
#scrape wiki for congresspeople
base_url='https://en.wikipedia.org/wiki/List_of_current_members_of_the_United_States_House_of_Representatives'
req=requests.get(base_url)
house_soup = BeautifulSoup(req.content, 'html.parser')
#create list of congresspeople
house=[house_soup.find_all('table')[6].find_all('tr')[i].find_all('td')[1].find_all('a')[-1]['title']
 for i in range(1,len(house_soup.find_all('table')[6].find_all('tr')))
 if len(house_soup.find_all('table')[6].find_all('tr')[i].find_all('td')[1].find_all('i'))==0]
#democratic candidates
demcands = ["Beto O'Rourke",'Pete Buttigieg','Julian Castro','Jay Inslee']
#concatenate political lists
politicians = [re.sub(" \(.*?\)","",name) for name in cabinet+senate+house+demcands]

In [15]:
#create list of articles
source_list = [df['articles'].iloc[i] for i in range(df.shape[0])]

In [16]:
#create dataframe of articles
art_df = pd.DataFrame([ast.literal_eval(source_list[i]) for i in range(len(source_list)) 
                       if type(source_list[i])!=float])

In [17]:
#convert published date to datetime
art_df['publishedAt'] = pd.to_datetime(art_df['publishedAt'])

In [18]:
#make sources consistent, single values
art_df['source_name']= [x['name'] for x in art_df['source']]

In [10]:

pols = list(art_df.columns[9:])
polmask=[]
for i in range(art_df.shape[0]):
    trues = sum(art_df.loc[i][pols])
    polmask.append(trues>0)

In [11]:
out_df=art_df[polmask][['content']]
out_df.dropna(subset=['content'],inplace=True)

In [36]:
out_df.to_csv('./articles_temp.csv')

In [37]:
rating_df=pd.read_csv('./train_data_rating.csv',header=None,names=['index','text','rating'])

In [38]:
rating_df.drop(labels='index',axis=1, inplace=True)

In [45]:
text = rating_df['text'].str.split("]",n=1,expand=True)
text.columns = ['id','article']
text['rating'] = rating_df['rating']

In [46]:
text['id'] = text['id'].str.replace("\[u","")
text['id'] = text['id'].str.replace("'","")
text['id'] = text['id'].str.replace("\[u'","")

In [47]:
ratings = text.groupby(['id','article']).mean()
ratings['rating']=ratings['rating'].round(0)
ratings.reset_index(inplace=True) 
ratings['article']=ratings['article'].str.replace("\[u'","")


In [48]:
ratings['negative'] = ratings['rating'].map({0:0,1:1,2:1,3:0,4:0,5:0})

In [49]:
ratings.to_csv('./ratings_temp.csv')

In [51]:
art_df.dropna(subset=['content'],inplace=True)

In [81]:
all_pols = politicians.append("Donald Trump")
art_df['subjects']=''
for subs in all_pols:
    print(subs)
    for row in art_df.index:
        if subs in art_df.loc[row]['content']:
            art_df.loc[row,'subjects'] = art_df.loc[row,'subjects']+" "+subs
            print(art_df.loc[row]['subjects'])
        elif row%1000 ==0:
            print(row)

Mike Pence
0
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
1000
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Penc

 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
15000
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
 Mike Pence
16000
Mike Pompeo
0
 Mike Pompeo
 Mike Pence Mike Pompeo
 Mike Pompeo
 Mike Pompeo
1000
 Mike Pompeo
 Mike Pompeo
 Mike Pompeo
 Mike Pompeo
 Mike Pompeo
 

 Chris Murphy
13000
 Chris Murphy
14000
 Chris Murphy
 Chris Murphy
15000
16000
Tom Carper
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Chris Coons
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Marco Rubio
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
 Mike Pompeo Marco Rubio
11000
12000
13000
14000
 Mike Pompeo Marco Rubio
15000
16000
Rick Scott
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Johnny Isakson
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
David Perdue
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Brian Schatz
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Mazie Hirono
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Mike Crapo
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Jim

 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mike Pence Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
13000
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
14000
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch McConnell
 Mitch M

9000
10000
11000
12000
13000
14000
15000
16000
Cory Booker
0
 Elizabeth Warren Cory Booker
 Kamala Harris Cory Booker
 Elizabeth Warren Cory Booker
 Kamala Harris Cory Booker
1000
 Cory Booker
 Elizabeth Warren Cory Booker
2000
 Kamala Harris Cory Booker
 Kamala Harris Cory Booker
 Elizabeth Warren Cory Booker
 Kamala Harris Cory Booker
 Cory Booker
 Elizabeth Warren Cory Booker
3000
 Kamala Harris Cory Booker
 Kamala Harris Cory Booker
 Cory Booker
 Cory Booker
 Kamala Harris Cory Booker
 Elizabeth Warren Cory Booker
 Kamala Harris Cory Booker
 Cory Booker
 Cory Booker
5000
 Elizabeth Warren Cory Booker
 Kamala Harris Cory Booker
 Elizabeth Warren Cory Booker
6000
 Elizabeth Warren Cory Booker
 Kamala Harris Cory Booker
 Cory Booker
 Elizabeth Warren Cory Booker
 Kamala Harris Cory Booker
7000
 Elizabeth Warren Cory Booker
 Kamala Harris Cory Booker
 Kamala Harris Cory Booker
 Elizabeth Warren Cory Booker
8000
 Cory Booker
 Kamala Harris Cory Booker
 Elizabeth Warren Cory Booker
9000


...

11000
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Nancy Pelosi Ilhan Omar
 Nancy Pelosi Ilhan Omar
12000
 Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Ilhan Omar
 Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Ilhan Omar
 Nancy Pelosi Ilhan Omar
13000
 Nancy Pelosi Ilhan Omar
 Ilhan Omar
 Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Rashida Tlaib Ilhan Omar
 Rashida Tlaib Ilhan Omar
 Ilhan Omar
 Ilhan Omar
 Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Ilhan Omar
 Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Ilhan Omar
14000
 Ilhan Omar
 Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Ilhan Omar
 Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Nancy Pelosi Ilhan Omar
 Nancy Pelosi Ilhan Omar
15000
 Nancy Pelosi Ilhan Omar
 Nancy Pelosi Ilhan Omar
16000
 Ayanna Pressley Rashida Tlaib Ilhan Omar
 Ayanna Pressley Rashida Tlaib Ilhan Omar
Tom 

 Alexandria Ocasio-Cortez
 Ayanna Pressley Rashida Tlaib Ilhan Omar Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Ayanna Pressley Rashida Tlaib Ilhan Omar Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Ayanna Pressley Rashida Tlaib Ilhan Omar Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Nancy Pelosi Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Ayanna Pressley Rashida Tlaib Ilhan Omar Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
6000
 Alexandria Ocasio-Cortez
 Ayanna Pressley Rashida Tlaib Ilhan Omar Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
7000
 Rashida Tlaib Alexandria Ocasio-Cortez
 Ayanna Pressley Rashida Tlaib Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Ayanna Pressley Rashida Tlaib Ilhan Omar Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
 Ayanna Pre

11000
 Tim Ryan
 Tim Ryan
12000
13000
14000
15000
 Tim Ryan
 Sherrod Brown Mike Turner Tim Ryan
 Tim Ryan
16000
David Joyce
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Steve Stivers
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Anthony Gonzalez
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Kevin Hern
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Markwayne Mullin
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Frank Lucas
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Tom Cole
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Kendra Horn
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Suzanne Bonamici
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Greg Walden

10000
11000
12000
13000
14000
15000
16000
Mac Thornberry
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Randy Weber
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Vicente González
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Veronica Escobar
0
1000
 Veronica Escobar
2000
3000
5000
 Mike Turner Veronica Escobar
6000
 Mike Turner Veronica Escobar
7000
8000
9000
 Veronica Escobar
10000
11000
12000
13000
14000
15000
 Veronica Escobar
16000
Bill Flores
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Sheila Jackson Lee
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Jodey Arrington
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Joaquin Castro
0
1000
2000
3000
 Joaquin Castro
5000
 Joaquin Castro
 Joaquin Castro
 Joaquin Castro
 Joaquin Castro
 Joaquin Castro
 Joaquin Castro

1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Mike Gallagher
0
1000
2000
3000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
Liz Cheney
0
1000
2000
3000
 Liz Cheney
 Liz Cheney
 Liz Cheney
5000
6000
7000
8000
9000
10000
11000
12000
13000
 Liz Cheney
 Liz Cheney
14000
15000
16000
Beto O'Rourke
0
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Steve King Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
1000
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
2000
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Amy Klobuchar Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
3000
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'Rourke
 Beto O'

In [91]:
db_df = art_df[['content' ,
'publishedAt' ,
'source_name',
'subjects']]
db_df=db_df[db_df['subjects']!='']

In [94]:
db_df.to_csv('./db_df.csv')